In [1]:
import datetime

import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras import Model, Sequential

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import MeanAbsoluteError

from tensorflow.keras.layers import Dense, Conv1D, LSTM, Lambda, Reshape, RNN, LSTMCell

import warnings
warnings.filterwarnings('ignore')

In [2]:
plt.rcParams['figure.figsize'] = (10, 7.5)
plt.rcParams['axes.grid'] = False

In [3]:
print(tf.__version__)

2.12.0


In [4]:
tf.random.set_seed(42)
np.random.seed(42)

In [7]:
df = pd.read_csv('data\BTC_final.csv')
df=df.drop('index',axis=1)
df.head()

,Date,Open,High,Low,Close,Volume,ScorePositive,ScoreNegative,ScoreNeutral
0,2022/1/1,46311.74609,47827.31250,46288.48438,47686.81250,2.458267e+10,0.035623,0.955734,0.008644
1,2022/1/2,47680.92578,47881.40625,46856.93750,47345.21875,2.795157e+10,0.035266,0.946176,0.008557
2,2022/1/3,47343.54297,47510.72656,45835.96484,46458.11719,3.307163e+10,0.163539,0.575673,0.260787
3,2022/1/4,46458.85156,47406.54688,45752.46484,45897.57422,4.249468e+10,0.260760,0.270590,0.468649
4,2022/1/5,45899.35938,46929.04688,42798.22266,43569.00391,3.685108e+10,0.321027,0.456918,0.222055


In [8]:
df.shape

(365, 9)

In [9]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Open,365.0,2.827869e+04,1.022322e+04,1.578230e+04,1.955047e+04,2.317953e+04,3.914845e+04,4.768093e+04
High,365.0,2.881781e+04,1.043534e+04,1.625305e+04,2.004109e+04,2.356383e+04,3.984593e+04,4.808684e+04
Low,365.0,2.761710e+04,9.970727e+03,1.559905e+04,1.917333e+04,2.272227e+04,3.811366e+04,4.710044e+04
Close,365.0,2.819775e+04,1.019804e+04,1.578728e+04,1.954685e+04,2.316463e+04,3.910515e+04,4.768681e+04
Volume,365.0,3.001326e+10,1.260657e+10,9.744636e+09,2.241721e+10,2.814822e+10,3.442156e+10,1.189920e+11
ScorePositive,365.0,1.715962e-01,1.186073e-01,1.046412e-02,7.798219e-02,1.626459e-01,2.383426e-01,9.247442e-01
ScoreNegative,365.0,4.536759e-01,1.919956e-01,9.944398e-03,3.399361e-01,4.564207e-01,5.682628e-01,9.645384e-01
ScoreNeutral,365.0,3.737416e-01,1.700731e-01,8.557184e-03,2.736989e-01,3.621676e-01,4.463143e-01,9.228845e-01


In [10]:
#將時間轉換為數值，以方便後續可以運算
timestamp_s = pd.to_datetime(df['Date']).map(datetime.datetime.timestamp)
df['Coin_timestamp'] = timestamp_s

In [13]:
# df = df.drop(['Date'], axis=1)
df.head()
df.to_csv('TimestampBTC.csv')

In [120]:
#二值化收盤價，昨天比今天高就是跌，就為False
# UpDown=[True]
# for i in range(1,len(df['Close'])):
#     if df['Close'][i]>df['Close'][i-1]:
#         UpDown.append(True)
#     else:
#         UpDown.append(False)
# df['UpDown'] = UpDown

In [13]:
#將資料集分為70%訓練、20%驗證、10%測試
n = len(df)
train_df = df[0:int(n*0.7)]
val_df = df[int(n*0.7):int(n*0.9)]
test_df = df[int(n*0.9):]

train_df.shape, val_df.shape, test_df.shape

((255, 9), (73, 9), (37, 9))

In [14]:
#將值縮限在0~1之間，加速訓練時間
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(train_df)

train_df[train_df.columns] = scaler.transform(train_df[train_df.columns])
val_df[val_df.columns] = scaler.transform(val_df[val_df.columns])
test_df[test_df.columns] = scaler.transform(test_df[test_df.columns])

In [15]:
train_df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Open,255.0,0.475870,0.324374,0.0,0.131029,0.561992,0.773590,1.0
High,255.0,0.482538,0.330615,0.0,0.130131,0.561767,0.790308,1.0
Low,255.0,0.477884,0.312671,0.0,0.140670,0.526522,0.757242,1.0
Close,255.0,0.472531,0.323701,0.0,0.130102,0.527627,0.772061,1.0
Volume,255.0,0.223899,0.140210,0.0,0.139343,0.209760,0.277317,1.0
ScorePositive,255.0,0.185835,0.139091,0.0,0.074279,0.177265,0.269369,1.0
ScoreNegative,255.0,0.460104,0.205251,0.0,0.342141,0.452662,0.583187,1.0
ScoreNeutral,255.0,0.395870,0.185510,0.0,0.296977,0.378257,0.473426,1.0
Coin_timestamp,255.0,0.500000,0.290379,0.0,0.250000,0.500000,0.750000,1.0


In [16]:
train_df.to_csv('data/train.csv')
val_df.to_csv('data/val.csv')
test_df.to_csv('data/test.csv')

In [17]:
train_df

,Open,High,Low,Close,Volume,ScorePositive,ScoreNegative,ScoreNeutral,Coin_timestamp
0,0.952530,0.990962,0.972375,1.000000,0.153933,0.027517,0.990777,0.000095,0.000000
1,1.000000,0.992846,0.991715,0.988159,0.201746,0.027128,0.980765,0.000000,0.003937
2,0.988303,0.979937,0.956979,0.957410,0.274412,0.167427,0.592638,0.276612,0.007874
3,0.957630,0.976309,0.954138,0.937980,0.408148,0.273763,0.273044,0.504568,0.011811
4,0.938233,0.959680,0.853625,0.857264,0.328052,0.339681,0.468234,0.234136,0.015748
...,...,...,...,...,...,...,...,...,...
250,0.015680,0.001588,0.046547,0.017060,0.261963,0.298499,0.259918,0.493507,0.984252
251,0.017004,0.072006,0.054517,0.088165,0.492946,0.434904,0.260025,0.356627,0.988189
252,0.088035,0.083180,0.117723,0.098543,0.328941,0.430441,0.257320,0.352967,0.992126
253,0.098493,0.083538,0.125828,0.101618,0.294598,0.033924,0.146048,0.877993,0.996063
